In [ ]:
import os
import fnmatch
import gc
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def processar_arquivos(diretorio_raiz, lista_epocas, tarefas, limite_por_evento=None, quantidade_pessoas=None):
    if not os.path.exists(diretorio_raiz):
        raise FileNotFoundError(f"O diretório raiz '{diretorio_raiz}' não foi encontrado.")

    pasta_pessoas = [p for p in os.listdir(diretorio_raiz) if os.path.isdir(os.path.join(diretorio_raiz, p))]
    if quantidade_pessoas is not None:
        pasta_pessoas = pasta_pessoas[:quantidade_pessoas]
    print("Quantidade de pessoas:", len(pasta_pessoas))

    canais_desejados = [
        'Fp1.', 'F7..', 'F3..', 'T7..', 'C3..', 'P7..', 'P3..', 'O1..',
        'Fp2.', 'F4..', 'F8..', 'C4..', 'T8..', 'P4..', 'P8..', 'O2..'
    ]

    epocas_por_evento = {evento: [] for evento in lista_epocas}
    total_arquivos_eventos = {evento: 0 for evento in lista_epocas}

    for pessoa in pasta_pessoas:
        pasta_pessoa = os.path.join(diretorio_raiz, pessoa)
        arquivos_edf = []
        for tarefa in tarefas:
            arquivos_edf += fnmatch.filter(os.listdir(pasta_pessoa), tarefa)

        raws = []
        for edf in arquivos_edf:
            raw = mne.io.read_raw_edf(os.path.join(pasta_pessoa, edf), preload=True)
            raw.pick_channels(canais_desejados)
            raws.append(raw)

        if raws:
            raw_concat = mne.concatenate_raws(raws)
            events, event_id = mne.events_from_annotations(raw_concat)
            for evento in lista_epocas:
                if evento in event_id:
                    epochs = mne.Epochs(
                        raw_concat, events,
                        event_id={evento: event_id[evento]},
                        tmin=-0.5, tmax=4, baseline=(-0.5, 0)
                    )
                    data = (epochs.get_data().astype(np.float32) * 1e6)  # volts→µV
                    epocas_por_evento[evento].extend(data)
                    total_arquivos_eventos[evento] += len(data)

    print("\nQuantidade de arquivos por evento:")
    for ev, cnt in total_arquivos_eventos.items():
        print(f"  {ev}: {cnt}")

    # definir limite_por_evento se não informado
    if limite_por_evento is None:
        limite_por_evento = int(np.mean([c for c in total_arquivos_eventos.values() if c > 0]))

    for ev, arr in epocas_por_evento.items():
        if len(arr) > limite_por_evento:
            np.random.shuffle(arr)
            epocas_por_evento[ev] = arr[:limite_por_evento]
            print(f"{ev} reduzido para {limite_por_evento}")

    return epocas_por_evento

def normaliza(tensor):
    normalized = []
    for sample in tensor:
        mn, mx = sample.min(), sample.max()
        normalized.append((sample - mn) / (mx - mn + 1e-8))
    return np.array(normalized)

def separar_dados(epocas_por_evento):
    arrays = {ev: np.array(eps) for ev, eps in epocas_por_evento.items()}
    data = np.concatenate([v for v in arrays.values() if v.size > 0])
    labels = []
    for idx, (ev, arr) in enumerate(arrays.items()):
        labels += [idx] * len(arr)
    x = normaliza(np.nan_to_num(data))
    y = np.array(labels)
    print("x shape:", x.shape, " y shape:", y.shape)
    return x, y

# Parâmetros e execução
diretorio = r"c:\Users\LaBios - BCI\Documents\eeg-motor-movementimagery-dataset-1.0.0\files"
eventos = ["T1", "T2"]
tarefas = ["*R04.edf", "*R08.edf", "*R12.edf"]

epocas = processar_arquivos(diretorio, eventos, tarefas, quantidade_pessoas=84)
x, y = separar_dados(epocas)
